In [1]:
from greedy_mcmc_attack import *
from data_collection.recordings import *
import sys
sys.path.append("../../")
from src.models.gcn import *
from src.models.gat import *
from src.models.gsage import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

trials = 5

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
edges_to_add = None

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model_save_path = "../models/cora_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges_full_run.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [7]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [8]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.806
Initial Loss: 0.655889093875885


In [9]:
G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.20)

In [10]:
full_metattack_acc, full_metattack_loss, full_metattack_itrs = [], [], []
for _ in range(trials):
    acc, loss, itrs = two_phase_attack_mcmc(data, train, model, 0.0, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)
    full_metattack_acc.append(acc)
    full_metattack_loss.append(loss)
    full_metattack_itrs.append(itrs)

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6068370938301086
Surrogate acc on unlabled data: 0.804
Attack loss: 0.30046606063842773
Surrogate loss on unlabled data: 0.7084909677505493
Surrogate acc on unlabled data: 0.777
Attack loss: 0.4210197925567627
Surrogate loss on unlabled data: 0.8470737934112549
Surrogate acc on unlabled data: 0.74
Attack loss: 0.5625224113464355
Surrogate loss on unlabled data: 0.8663991093635559
Surrogate acc on unlabled data: 0.722
Attack loss: 0.603731095790863
Surrogate loss on unlabled data: 0.9413387179374695
Surrogate acc on unlabled data: 0.705
Attack loss: 0.674537181854248
Surrogate loss on unlabled data: 1.0730719566345215
Surrogate acc on unlabled data: 0.674
Attack loss: 0.8091936111450195
Surrogate loss on unlabled data: 1.1026626825332642
Surrogate acc on unlabled data: 0.667
Attack loss: 0.850286602973938
Surrogate loss on unlabled data: 1.2550314664840698
Surrogate acc on unlabled data: 0.633
Attack loss: 0.9847468137741089
Surrogate loss on unlabled 

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5997056365013123
Surrogate acc on unlabled data: 0.801
Attack loss: 0.2820113003253937
Surrogate loss on unlabled data: 0.6891105771064758
Surrogate acc on unlabled data: 0.778
Attack loss: 0.3972746729850769
Surrogate loss on unlabled data: 0.8251105546951294
Surrogate acc on unlabled data: 0.756
Attack loss: 0.5475964546203613
Surrogate loss on unlabled data: 0.9452724456787109
Surrogate acc on unlabled data: 0.721
Attack loss: 0.6511635780334473
Surrogate loss on unlabled data: 1.0572682619094849
Surrogate acc on unlabled data: 0.68
Attack loss: 0.747749388217926
Surrogate loss on unlabled data: 1.0714712142944336
Surrogate acc on unlabled data: 0.6910000000000001
Attack loss: 0.8016696572303772
Surrogate loss on unlabled data: 1.1499443054199219
Surrogate acc on unlabled data: 0.654
Attack loss: 0.8966174125671387
Surrogate loss on unlabled data: 1.287614107131958
Surrogate acc on unlabled data: 0.628
Attack loss: 1.026801586151123
Surrogate loss 

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.6084439158439636
Surrogate acc on unlabled data: 0.794
Attack loss: 0.3024390935897827
Surrogate loss on unlabled data: 0.7434649467468262
Surrogate acc on unlabled data: 0.774
Attack loss: 0.416003942489624
Surrogate loss on unlabled data: 0.7718805074691772
Surrogate acc on unlabled data: 0.752
Attack loss: 0.48842594027519226
Surrogate loss on unlabled data: 0.9251621961593628
Surrogate acc on unlabled data: 0.715
Attack loss: 0.6574955582618713
Surrogate loss on unlabled data: 0.9905140995979309
Surrogate acc on unlabled data: 0.6970000000000001
Attack loss: 0.729589581489563
Surrogate loss on unlabled data: 1.0510040521621704
Surrogate acc on unlabled data: 0.681
Attack loss: 0.785953938961029
Surrogate loss on unlabled data: 1.091080665588379
Surrogate acc on unlabled data: 0.665
Attack loss: 0.8794097304344177
Surrogate loss on unlabled data: 1.1882089376449585
Surrogate acc on unlabled data: 0.628
Attack loss: 0.9597114324569702
Surrogate loss

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6020966172218323
Surrogate acc on unlabled data: 0.806
Attack loss: 0.28400111198425293
Surrogate loss on unlabled data: 0.7171956300735474
Surrogate acc on unlabled data: 0.778
Attack loss: 0.4195323884487152
Surrogate loss on unlabled data: 0.84706050157547
Surrogate acc on unlabled data: 0.755
Attack loss: 0.5221629738807678
Surrogate loss on unlabled data: 0.8205362558364868
Surrogate acc on unlabled data: 0.759
Attack loss: 0.5541848540306091
Surrogate loss on unlabled data: 0.8834395408630371
Surrogate acc on unlabled data: 0.717
Attack loss: 0.656029462814331
Surrogate loss on unlabled data: 1.0235694646835327
Surrogate acc on unlabled data: 0.686
Attack loss: 0.8050374984741211
Surrogate loss on unlabled data: 1.1882277727127075
Surrogate acc on unlabled data: 0.651
Attack loss: 0.9571371078491211
Surrogate loss on unlabled data: 1.2400418519973755
Surrogate acc on unlabled data: 0.631
Attack loss: 1.013958215713501
Surrogate loss on unlabled 

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5897972583770752
Surrogate acc on unlabled data: 0.81
Attack loss: 0.28807327151298523
Surrogate loss on unlabled data: 0.7158804535865784
Surrogate acc on unlabled data: 0.777
Attack loss: 0.4385462701320648
Surrogate loss on unlabled data: 0.8135526776313782
Surrogate acc on unlabled data: 0.757
Attack loss: 0.533322274684906
Surrogate loss on unlabled data: 0.8907017707824707
Surrogate acc on unlabled data: 0.723
Attack loss: 0.6289617419242859
Surrogate loss on unlabled data: 0.999080240726471
Surrogate acc on unlabled data: 0.709
Attack loss: 0.7437759637832642
Surrogate loss on unlabled data: 1.1107720136642456
Surrogate acc on unlabled data: 0.6920000000000001
Attack loss: 0.8190533518791199
Surrogate loss on unlabled data: 1.2060866355895996
Surrogate acc on unlabled data: 0.646
Attack loss: 0.9737642407417297
Surrogate loss on unlabled data: 1.1488673686981201
Surrogate acc on unlabled data: 0.629
Attack loss: 0.9476047158241272
Surrogate los

# regular, secondary attack, not reversed

In [11]:
splits = [0.0, 0.5]
split_dic_acc = defaultdict(lambda: defaultdict(list))
split_dic_loss = defaultdict(lambda: defaultdict(list))
itrs_one = defaultdict(lambda: defaultdict(int))

In [13]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_acc[s][j], split_dic_loss[s][j], itrs_one[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, binary_fn, constant_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6012418270111084
Surrogate acc on unlabled data: 0.806
Attack loss: 0.2838417887687683
Surrogate loss on unlabled data: 0.6705977320671082
Surrogate acc on unlabled data: 0.79
Attack loss: 0.41419318318367004
Surrogate loss on unlabled data: 0.7803844809532166
Surrogate acc on unlabled data: 0.763
Attack loss: 0.5256519913673401
Surrogate loss on unlabled data: 0.8685051798820496
Surrogate acc on unlabled data: 0.73
Attack loss: 0.6215494871139526
Surrogate loss on unlabled data: 0.9911563992500305
Surrogate acc on unlabled data: 0.705
Attack loss: 0.7090355157852173
Surrogate loss on unlabled data: 1.0920027494430542
Surrogate acc on unlabled data: 0.682
Attack loss: 0.8023767471313477
Surrogate loss on unlabled data: 1.204359769821167
Surrogate acc on unlabled data: 0.648
Attack loss: 0.9189240336418152
Surrogate loss on unlabled data: 1.1870393753051758
Surrogate acc on unlabled data: 0.649
Attack loss: 0.9301344752311707
Surrogate loss on unlabled

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.7575885057449341
Surrogate acc on unlabled data: 0.767
Attack loss: 0.42350056767463684
Surrogate loss on unlabled data: 0.8347712755203247
Surrogate acc on unlabled data: 0.741
Attack loss: 0.5323217511177063
Surrogate loss on unlabled data: 0.9180708527565002
Surrogate acc on unlabled data: 0.708
Attack loss: 0.6416742205619812
Surrogate loss on unlabled data: 0.9826439023017883
Surrogate acc on unlabled data: 0.6920000000000001
Attack loss: 0.7455166578292847
Surrogate loss on unlabled data: 1.1982780694961548
Surrogate acc on unlabled data: 0.618
Attack loss: 0.9592316150665283
Surrogate loss on unlabled data: 1.1735529899597168
Surrogate acc on unlabled data: 0.606
Attack loss: 0.93509840965271
Surrogate loss on unlabled data: 1.4467014074325562
Surrogate acc on unlabled data: 0.5670000000000001
Attack loss: 1.1906147003173828
Surrogate loss on unlabled data: 1.5753390789031982
Surrogate acc on unlabled data: 0.548
Attack loss: 1.3248043060302734

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.7035601139068604
Surrogate acc on unlabled data: 0.766
Attack loss: 0.3935573995113373
Surrogate loss on unlabled data: 0.8588764071464539
Surrogate acc on unlabled data: 0.726
Attack loss: 0.597614586353302
Surrogate loss on unlabled data: 0.95952308177948
Surrogate acc on unlabled data: 0.687
Attack loss: 0.7084372639656067
Surrogate loss on unlabled data: 1.027892827987671
Surrogate acc on unlabled data: 0.673
Attack loss: 0.7861806154251099
Surrogate loss on unlabled data: 1.1670666933059692
Surrogate acc on unlabled data: 0.621
Attack loss: 0.9355140328407288
Surrogate loss on unlabled data: 1.3095992803573608
Surrogate acc on unlabled data: 0.593
Attack loss: 1.099209189414978
Surrogate loss on unlabled data: 1.3767129182815552
Surrogate acc on unlabled data: 0.603
Attack loss: 1.1564191579818726
Surrogate loss on unlabled data: 1.5148096084594727
Surrogate acc on unlabled data: 0.5700000000000001
Attack loss: 1.290480375289917
Surrogate loss on

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.597913920879364
Surrogate acc on unlabled data: 0.807
Attack loss: 0.2830425500869751
Surrogate loss on unlabled data: 0.6944217681884766
Surrogate acc on unlabled data: 0.771
Attack loss: 0.42795151472091675
Surrogate loss on unlabled data: 0.7707327008247375
Surrogate acc on unlabled data: 0.756
Attack loss: 0.5278706550598145
Surrogate loss on unlabled data: 0.912020206451416
Surrogate acc on unlabled data: 0.72
Attack loss: 0.6452034115791321
Surrogate loss on unlabled data: 0.9630037546157837
Surrogate acc on unlabled data: 0.685
Attack loss: 0.7332802414894104
Surrogate loss on unlabled data: 1.0958497524261475
Surrogate acc on unlabled data: 0.667
Attack loss: 0.8798996806144714
Surrogate loss on unlabled data: 1.175622582435608
Surrogate acc on unlabled data: 0.65
Attack loss: 0.9547324180603027
Surrogate loss on unlabled data: 1.2370208501815796
Surrogate acc on unlabled data: 0.614
Attack loss: 1.0368058681488037
Surrogate loss on unlabled d

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.784133791923523
Surrogate acc on unlabled data: 0.756
Attack loss: 0.42156392335891724
Surrogate loss on unlabled data: 0.799956738948822
Surrogate acc on unlabled data: 0.748
Attack loss: 0.5190451741218567
Surrogate loss on unlabled data: 0.9469319581985474
Surrogate acc on unlabled data: 0.704
Attack loss: 0.6746019124984741
Surrogate loss on unlabled data: 1.0026143789291382
Surrogate acc on unlabled data: 0.68
Attack loss: 0.7459326386451721
Surrogate loss on unlabled data: 1.1721093654632568
Surrogate acc on unlabled data: 0.635
Attack loss: 0.9088451266288757
Surrogate loss on unlabled data: 1.2146955728530884
Surrogate acc on unlabled data: 0.626
Attack loss: 0.9762099981307983
Surrogate loss on unlabled data: 1.3212201595306396
Surrogate acc on unlabled data: 0.587
Attack loss: 1.0987157821655273
Surrogate loss on unlabled data: 1.539193034172058
Surrogate acc on unlabled data: 0.558
Attack loss: 1.3094977140426636
Surrogate loss on unlabled 

In [14]:
record1 = Recording(dict(split_dic_loss), dict(split_dic_acc), dict(itrs_one), Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"constant"}_{"binary"}_{"attacked"}_results.pkl')

In [15]:
# plot_results(split_dic_acc, ptb_rate, "MCMC", "binary", "constant", "accuracy", "regular")

In [16]:
# plot_results(split_dic_loss, ptb_rate, "MCMC", "binary", "constant", "loss", "regular")

##### ideally, we could show some metric, where (change in loss after hidden for %) > (% * max change in loss)

In [17]:
split_dic_decreasing_acc = defaultdict(lambda: defaultdict(list))
itrs_two = defaultdict(lambda: defaultdict(int))
split_dic_decreasing_loss = defaultdict(lambda: defaultdict(list))

In [18]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_acc[s][j], split_dic_decreasing_loss[s][j], itrs_two[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, binary_fn, increasing_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5931676626205444
Surrogate acc on unlabled data: 0.8
Attack loss: 0.28783389925956726
Surrogate loss on unlabled data: 0.6664612293243408
Surrogate acc on unlabled data: 0.781
Attack loss: 0.41207513213157654
Surrogate loss on unlabled data: 0.7459198832511902
Surrogate acc on unlabled data: 0.773
Attack loss: 0.4845704734325409
Surrogate loss on unlabled data: 0.8716647624969482
Surrogate acc on unlabled data: 0.733
Attack loss: 0.6199064254760742
Surrogate loss on unlabled data: 0.9686453342437744
Surrogate acc on unlabled data: 0.714
Attack loss: 0.6982962489128113
Surrogate loss on unlabled data: 1.0000560283660889
Surrogate acc on unlabled data: 0.6900000000000001
Attack loss: 0.7526084184646606
Surrogate loss on unlabled data: 1.1231212615966797
Surrogate acc on unlabled data: 0.663
Attack loss: 0.8916213512420654
Surrogate loss on unlabled data: 1.1981892585754395
Surrogate acc on unlabled data: 0.635
Attack loss: 0.9483942985534668
Surrogate l

Peturbing graph...:   0%|          | 0/274 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.7098289132118225
Surrogate acc on unlabled data: 0.777
Attack loss: 0.36278480291366577
Surrogate loss on unlabled data: 0.7909355759620667
Surrogate acc on unlabled data: 0.752
Attack loss: 0.503136157989502
Surrogate loss on unlabled data: 0.9438632726669312
Surrogate acc on unlabled data: 0.705
Attack loss: 0.6175724267959595
Surrogate loss on unlabled data: 1.0465543270111084
Surrogate acc on unlabled data: 0.653
Attack loss: 0.7626539468765259
Surrogate loss on unlabled data: 1.1924394369125366
Surrogate acc on unlabled data: 0.614
Attack loss: 0.9145048260688782
Surrogate loss on unlabled data: 1.3291945457458496
Surrogate acc on unlabled data: 0.582
Attack loss: 1.0657539367675781
Surrogate loss on unlabled data: 1.2436503171920776
Surrogate acc on unlabled data: 0.5730000000000001
Attack loss: 0.9972237944602966
Surrogate loss on unlabled data: 1.5884974002838135
Surrogate acc on unlabled data: 0.556
Attack loss: 1.3187146186828613
Surrogate l

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.6676307320594788
Surrogate acc on unlabled data: 0.781
Attack loss: 0.3527027368545532
Surrogate loss on unlabled data: 0.8092043995857239
Surrogate acc on unlabled data: 0.752
Attack loss: 0.5177754759788513
Surrogate loss on unlabled data: 0.9377474784851074
Surrogate acc on unlabled data: 0.7020000000000001
Attack loss: 0.6291710734367371
Surrogate loss on unlabled data: 0.951970100402832
Surrogate acc on unlabled data: 0.6890000000000001
Attack loss: 0.692419171333313
Surrogate loss on unlabled data: 1.0814422369003296
Surrogate acc on unlabled data: 0.666
Attack loss: 0.8172567486763
Surrogate loss on unlabled data: 1.1378107070922852
Surrogate acc on unlabled data: 0.618
Attack loss: 0.8944934010505676
Surrogate loss on unlabled data: 1.3239750862121582
Surrogate acc on unlabled data: 0.5730000000000001
Attack loss: 1.102909803390503
Surrogate loss on unlabled data: 1.4931726455688477
Surrogate acc on unlabled data: 0.549
Attack loss: 1.29580104

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6047163605690002
Surrogate acc on unlabled data: 0.807
Attack loss: 0.2842140793800354
Surrogate loss on unlabled data: 0.6893631219863892
Surrogate acc on unlabled data: 0.781
Attack loss: 0.3984927237033844
Surrogate loss on unlabled data: 0.8248196244239807
Surrogate acc on unlabled data: 0.757
Attack loss: 0.5469117164611816
Surrogate loss on unlabled data: 0.8709099292755127
Surrogate acc on unlabled data: 0.732
Attack loss: 0.6174976825714111
Surrogate loss on unlabled data: 0.9850638508796692
Surrogate acc on unlabled data: 0.685
Attack loss: 0.7655852437019348
Surrogate loss on unlabled data: 1.0446463823318481
Surrogate acc on unlabled data: 0.678
Attack loss: 0.8208886981010437
Surrogate loss on unlabled data: 1.1002874374389648
Surrogate acc on unlabled data: 0.641
Attack loss: 0.8699501752853394
Surrogate loss on unlabled data: 1.2917985916137695
Surrogate acc on unlabled data: 0.621
Attack loss: 1.0857245922088623
Surrogate loss on unlabl

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6895871162414551
Surrogate acc on unlabled data: 0.78
Attack loss: 0.362382709980011
Surrogate loss on unlabled data: 0.8278078436851501
Surrogate acc on unlabled data: 0.746
Attack loss: 0.5480291247367859
Surrogate loss on unlabled data: 0.9446468949317932
Surrogate acc on unlabled data: 0.708
Attack loss: 0.6281259655952454
Surrogate loss on unlabled data: 0.9996041059494019
Surrogate acc on unlabled data: 0.665
Attack loss: 0.750416100025177
Surrogate loss on unlabled data: 1.1176899671554565
Surrogate acc on unlabled data: 0.629
Attack loss: 0.8632440567016602
Surrogate loss on unlabled data: 1.1902682781219482
Surrogate acc on unlabled data: 0.59
Attack loss: 0.9690746068954468
Surrogate loss on unlabled data: 1.3756660223007202
Surrogate acc on unlabled data: 0.587
Attack loss: 1.137825608253479
Surrogate loss on unlabled data: 1.4903422594070435
Surrogate acc on unlabled data: 0.56
Attack loss: 1.2603892087936401
Surrogate loss on unlabled dat

In [19]:
record1 = Recording(dict(split_dic_decreasing_loss), dict(split_dic_decreasing_acc), dict(itrs_two), Model.GCN, Dataset.CORA, AcceptFn.INCREASING, SelectFn.BINARY, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"increasing"}_{"binary"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_acc, ptb_rate, "MCMC", "binary", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_loss, ptb_rate, "MCMC", "binary", "increasing", "loss", "regular")

In [20]:
split_dic_decreasing_constant_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_constant_loss = defaultdict(lambda: defaultdict(list))
itrs_three = defaultdict(lambda: defaultdict(int))

In [21]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_constant_acc[s][j], split_dic_decreasing_constant_loss[s][j], itrs_three[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, decreasing_fn, constant_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.632856547832489
Surrogate acc on unlabled data: 0.799
Attack loss: 0.2835509479045868
Surrogate loss on unlabled data: 0.7193488478660583
Surrogate acc on unlabled data: 0.783
Attack loss: 0.41393569111824036
Surrogate loss on unlabled data: 0.7924701571464539
Surrogate acc on unlabled data: 0.761
Attack loss: 0.5370346903800964
Surrogate loss on unlabled data: 0.9276283979415894
Surrogate acc on unlabled data: 0.721
Attack loss: 0.6435145139694214
Surrogate loss on unlabled data: 0.9846339225769043
Surrogate acc on unlabled data: 0.7030000000000001
Attack loss: 0.717789351940155
Surrogate loss on unlabled data: 1.050170660018921
Surrogate acc on unlabled data: 0.681
Attack loss: 0.8020492792129517
Surrogate loss on unlabled data: 1.0707201957702637
Surrogate acc on unlabled data: 0.667
Attack loss: 0.8481172919273376
Surrogate loss on unlabled data: 1.2164816856384277
Surrogate acc on unlabled data: 0.643
Attack loss: 0.9805523753166199
Surrogate los

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6813849806785583
Surrogate acc on unlabled data: 0.775
Attack loss: 0.3890857398509979
Surrogate loss on unlabled data: 0.8252817988395691
Surrogate acc on unlabled data: 0.731
Attack loss: 0.5764976143836975
Surrogate loss on unlabled data: 0.9530894756317139
Surrogate acc on unlabled data: 0.712
Attack loss: 0.7014861702919006
Surrogate loss on unlabled data: 0.9737381339073181
Surrogate acc on unlabled data: 0.712
Attack loss: 0.7686940431594849
Surrogate loss on unlabled data: 1.0579129457473755
Surrogate acc on unlabled data: 0.659
Attack loss: 0.8555724620819092
Surrogate loss on unlabled data: 1.1951892375946045
Surrogate acc on unlabled data: 0.618
Attack loss: 0.9774240255355835
Surrogate loss on unlabled data: 1.2683675289154053
Surrogate acc on unlabled data: 0.591
Attack loss: 1.0694040060043335
Surrogate loss on unlabled data: 1.3970904350280762
Surrogate acc on unlabled data: 0.582
Attack loss: 1.2369693517684937
Surrogate loss on unlabl

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.6842619776725769
Surrogate acc on unlabled data: 0.78
Attack loss: 0.3805081248283386
Surrogate loss on unlabled data: 0.8002855181694031
Surrogate acc on unlabled data: 0.749
Attack loss: 0.5062248706817627
Surrogate loss on unlabled data: 0.8767186999320984
Surrogate acc on unlabled data: 0.725
Attack loss: 0.6016688346862793
Surrogate loss on unlabled data: 1.0008095502853394
Surrogate acc on unlabled data: 0.704
Attack loss: 0.7329755425453186
Surrogate loss on unlabled data: 1.0843757390975952
Surrogate acc on unlabled data: 0.681
Attack loss: 0.8501830101013184
Surrogate loss on unlabled data: 1.2307133674621582
Surrogate acc on unlabled data: 0.629
Attack loss: 0.9936796426773071
Surrogate loss on unlabled data: 1.3428940773010254
Surrogate acc on unlabled data: 0.601
Attack loss: 1.1221768856048584
Surrogate loss on unlabled data: 1.507953405380249
Surrogate acc on unlabled data: 0.582
Attack loss: 1.2980172634124756
Surrogate loss on unlabled

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.6005353331565857
Surrogate acc on unlabled data: 0.805
Attack loss: 0.3043343424797058
Surrogate loss on unlabled data: 0.6762751340866089
Surrogate acc on unlabled data: 0.782
Attack loss: 0.3933180868625641
Surrogate loss on unlabled data: 0.7764924168586731
Surrogate acc on unlabled data: 0.756
Attack loss: 0.48561790585517883
Surrogate loss on unlabled data: 0.862816333770752
Surrogate acc on unlabled data: 0.732
Attack loss: 0.5996371507644653
Surrogate loss on unlabled data: 1.0259870290756226
Surrogate acc on unlabled data: 0.686
Attack loss: 0.7825993299484253
Surrogate loss on unlabled data: 1.0786726474761963
Surrogate acc on unlabled data: 0.675
Attack loss: 0.8473739624023438
Surrogate loss on unlabled data: 1.079145073890686
Surrogate acc on unlabled data: 0.667
Attack loss: 0.847186803817749
Surrogate loss on unlabled data: 1.3289320468902588
Surrogate acc on unlabled data: 0.637
Attack loss: 1.0861881971359253
Surrogate loss on unlabled

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.7181274890899658
Surrogate acc on unlabled data: 0.775
Attack loss: 0.40189412236213684
Surrogate loss on unlabled data: 0.8355819582939148
Surrogate acc on unlabled data: 0.752
Attack loss: 0.5529728531837463
Surrogate loss on unlabled data: 0.9664552211761475
Surrogate acc on unlabled data: 0.6980000000000001
Attack loss: 0.685973584651947
Surrogate loss on unlabled data: 1.0092216730117798
Surrogate acc on unlabled data: 0.661
Attack loss: 0.7365323901176453
Surrogate loss on unlabled data: 1.1685479879379272
Surrogate acc on unlabled data: 0.617
Attack loss: 0.9005363583564758
Surrogate loss on unlabled data: 1.2951310873031616
Surrogate acc on unlabled data: 0.591
Attack loss: 1.0642726421356201
Surrogate loss on unlabled data: 1.2903579473495483
Surrogate acc on unlabled data: 0.581
Attack loss: 1.0325454473495483
Surrogate loss on unlabled data: 1.571388840675354
Surrogate acc on unlabled data: 0.544
Attack loss: 1.3176416158676147
Surrogate lo

In [22]:
record1 = Recording(dict(split_dic_decreasing_constant_loss), dict(split_dic_decreasing_constant_acc), dict(itrs_three), Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"constant"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_constant_acc, ptb_rate, "MCMC", "decaying", "constant", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_constant_loss, ptb_rate, "MCMC", "decaying", "constant", "loss", "regular")

In [23]:
split_dic_decreasing_gradual_acc = defaultdict(lambda: defaultdict(list))
split_dic_decreasing_gradual_loss = defaultdict(lambda: defaultdict(list))
itrs_four = defaultdict(lambda: defaultdict(int))

In [24]:
for s in splits:
    for j in range(trials):
        if s == 0.0:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = full_metattack_acc[j], full_metattack_loss[j], full_metattack_itrs[j]
        else:
            split_dic_decreasing_gradual_acc[s][j], split_dic_decreasing_gradual_loss[s][j], itrs_four[s][j] = two_phase_attack_mcmc(data, train, model, s, edges_to_add, decreasing_fn, increasing_fn, device, is_reversed=False, verbose=True)

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.5982744693756104
Surrogate acc on unlabled data: 0.802
Attack loss: 0.28062403202056885
Surrogate loss on unlabled data: 0.6896297931671143
Surrogate acc on unlabled data: 0.782
Attack loss: 0.40674692392349243
Surrogate loss on unlabled data: 0.7658402919769287
Surrogate acc on unlabled data: 0.756
Attack loss: 0.48320046067237854
Surrogate loss on unlabled data: 0.9583722949028015
Surrogate acc on unlabled data: 0.717
Attack loss: 0.6828652024269104
Surrogate loss on unlabled data: 0.9987233281135559
Surrogate acc on unlabled data: 0.712
Attack loss: 0.7348046898841858
Surrogate loss on unlabled data: 1.1195052862167358
Surrogate acc on unlabled data: 0.672
Attack loss: 0.8333209156990051
Surrogate loss on unlabled data: 1.187225103378296
Surrogate acc on unlabled data: 0.665
Attack loss: 0.9417396187782288
Surrogate loss on unlabled data: 1.3121904134750366
Surrogate acc on unlabled data: 0.637
Attack loss: 1.08322012424469
Surrogate loss on unlabl

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.7348129749298096
Surrogate acc on unlabled data: 0.753
Attack loss: 0.4105481207370758
Surrogate loss on unlabled data: 0.862910807132721
Surrogate acc on unlabled data: 0.724
Attack loss: 0.5495224595069885
Surrogate loss on unlabled data: 0.9388043284416199
Surrogate acc on unlabled data: 0.686
Attack loss: 0.6388145685195923
Surrogate loss on unlabled data: 1.1082745790481567
Surrogate acc on unlabled data: 0.641
Attack loss: 0.8430572748184204
Surrogate loss on unlabled data: 1.15062415599823
Surrogate acc on unlabled data: 0.63
Attack loss: 0.9038282036781311
Surrogate loss on unlabled data: 1.3028675317764282
Surrogate acc on unlabled data: 0.594
Attack loss: 1.0712430477142334
Surrogate loss on unlabled data: 1.3940622806549072
Surrogate acc on unlabled data: 0.586
Attack loss: 1.1466068029403687
Surrogate loss on unlabled data: 1.49863600730896
Surrogate acc on unlabled data: 0.5680000000000001
Attack loss: 1.2692986726760864
Surrogate loss on

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

...

Surrogate loss on unlabled data: 0.7208777666091919
Surrogate acc on unlabled data: 0.755
Attack loss: 0.44347015023231506
Surrogate loss on unlabled data: 0.8577008843421936
Surrogate acc on unlabled data: 0.72
Attack loss: 0.602658748626709
Surrogate loss on unlabled data: 0.9494959115982056
Surrogate acc on unlabled data: 0.682
Attack loss: 0.7047126293182373
Surrogate loss on unlabled data: 1.010054588317871
Surrogate acc on unlabled data: 0.67
Attack loss: 0.7943087816238403
Surrogate loss on unlabled data: 1.118809461593628
Surrogate acc on unlabled data: 0.638
Attack loss: 0.9335593581199646
Surrogate loss on unlabled data: 1.2090193033218384
Surrogate acc on unlabled data: 0.6
Attack loss: 1.0188086032867432
Surrogate loss on unlabled data: 1.4259231090545654
Surrogate acc on unlabled data: 0.579
Attack loss: 1.2506415843963623
Surrogate loss on unlabled data: 1.438476324081421
Surrogate acc on unlabled data: 0.5760000000000001
Attack loss: 1.303356409072876
Surrogate loss on u

Peturbing graph...:   0%|          | 0/263 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.610002338886261
Surrogate acc on unlabled data: 0.8130000000000001
Attack loss: 0.28099387884140015
Surrogate loss on unlabled data: 0.7336834669113159
Surrogate acc on unlabled data: 0.771
Attack loss: 0.4555664658546448
Surrogate loss on unlabled data: 0.8008192777633667
Surrogate acc on unlabled data: 0.759
Attack loss: 0.5416727066040039
Surrogate loss on unlabled data: 0.9226359724998474
Surrogate acc on unlabled data: 0.727
Attack loss: 0.6400845050811768
Surrogate loss on unlabled data: 1.0191385746002197
Surrogate acc on unlabled data: 0.7020000000000001
Attack loss: 0.7506475448608398
Surrogate loss on unlabled data: 1.1078906059265137
Surrogate acc on unlabled data: 0.677
Attack loss: 0.8446716666221619
Surrogate loss on unlabled data: 1.2228537797927856
Surrogate acc on unlabled data: 0.669
Attack loss: 0.9686131477355957
Surrogate loss on unlabled data: 1.2232646942138672
Surrogate acc on unlabled data: 0.656
Attack loss: 0.972196459770202

Peturbing graph...:   0%|          | 0/275 [00:00<?, ?it/s]

Surrogate loss on unlabled data: 0.68674635887146
Surrogate acc on unlabled data: 0.773
Attack loss: 0.3842950165271759
Surrogate loss on unlabled data: 0.8536203503608704
Surrogate acc on unlabled data: 0.723
Attack loss: 0.5589499473571777
Surrogate loss on unlabled data: 0.9957864880561829
Surrogate acc on unlabled data: 0.6940000000000001
Attack loss: 0.7322757244110107
Surrogate loss on unlabled data: 1.062535285949707
Surrogate acc on unlabled data: 0.661
Attack loss: 0.7998966574668884
Surrogate loss on unlabled data: 1.1187039613723755
Surrogate acc on unlabled data: 0.634
Attack loss: 0.8833587765693665
Surrogate loss on unlabled data: 1.1426777839660645
Surrogate acc on unlabled data: 0.614
Attack loss: 0.9187888503074646
Surrogate loss on unlabled data: 1.4581329822540283
Surrogate acc on unlabled data: 0.585
Attack loss: 1.243511438369751
Surrogate loss on unlabled data: 1.4105240106582642
Surrogate acc on unlabled data: 0.578
Attack loss: 1.1958565711975098
Surrogate loss 

In [25]:
record1 = Recording(dict(split_dic_decreasing_gradual_loss), dict(split_dic_decreasing_gradual_acc), dict(itrs_four), Model.GCN, Dataset.CORA, AcceptFn.CONSTANT, SelectFn.DECAYING, Reverse.ATTACKED)
record1.save(f'data_collection/{"mcmc"}_{"gcn"}_{"cora"}_{"increasing"}_{"decaying"}_{"attacked"}_results.pkl')

In [ ]:
# plot_results(split_dic_decreasing_gradual_acc, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")

In [ ]:
# plot_results(split_dic_decreasing_gradual_loss, ptb_rate, "MCMC", "decaying", "increasing", "accuracy", "regular")